# Data Preparation for the SOSTAT 2021 Tutorials


## Imports

In [1]:
# make plots interactive and import plotting functionality
%matplotlib notebook
import matplotlib.pyplot as plt

# pretty plotting
import seaborn as sns

# my standard styles for plots
sns.set_style("whitegrid")
sns.set_context("talk")

# Always need numpy
import numpy as np

# data array operations
import pandas as pd

In [2]:
from astroquery.sdss import SDSS

In [4]:
stars = SDSS.query_sql("""SELECT * from (SELECT top 50000
objid, ra, dec, psfmag_u-extinction_u AS mag_u,
psfmag_g-extinction_g AS mag_g, psfmag_r-extinction_r AS mag_r, psfmag_i-extinction_i AS mag_i,psfmag_z-extinction_z AS mag_z,z AS spec_z,dered_u - dered_g AS u_g_color, 
dered_g - dered_r AS g_r_color,dered_r - dered_i AS r_i_color,dered_i - dered_z AS i_z_color,class
FROM SpecPhoto 
WHERE 
 (class = 'STAR')
 ) as sp
""")

In [5]:
galaxies = SDSS.query_sql("""SELECT * from (SELECT top 50000
objid, ra, dec, psfmag_u-extinction_u AS mag_u,
psfmag_g-extinction_g AS mag_g, psfmag_r-extinction_r AS mag_r, psfmag_i-extinction_i AS mag_i,psfmag_z-extinction_z AS mag_z,z AS spec_z,dered_u - dered_g AS u_g_color, 
dered_g - dered_r AS g_r_color,dered_r - dered_i AS r_i_color,dered_i - dered_z AS i_z_color,class
FROM SpecPhoto 
WHERE 
 (class = 'GALAXY')
 ) as sp
""")

In [6]:
stars = stars.to_pandas()
stars["class"] = np.array(stars["class"], dtype=str)


In [7]:
galaxies = galaxies.to_pandas()
galaxies["class"] = np.array(galaxies["class"], dtype=str)


In [9]:
comb = pd.concat([stars[:10000], galaxies[:10000]])

In [11]:
comb.head()

,objid,ra,dec,mag_u,mag_g,mag_r,mag_i,mag_z,spec_z,u_g_color,g_r_color,r_i_color,i_z_color,class
0,1237655499736023309,198.75865,-1.661133,19.28792,17.36450,16.65784,16.41547,16.26836,0.000057,1.881269,0.760563,0.201073,0.206329,STAR
1,1237655495977468550,198.69212,-1.551051,23.98161,21.48122,20.69408,19.68314,19.05251,-0.000028,2.914644,0.825191,0.962955,0.646452,STAR
2,1237655495977533617,198.86527,-1.521170,18.74677,17.72853,17.71136,17.71290,17.87158,0.000351,1.020382,0.024298,-0.013058,-0.148605,STAR
3,1237655495977533557,198.79422,-1.477119,19.93713,18.80160,18.82963,18.94912,19.04007,0.000199,1.160055,-0.032026,-0.119993,-0.069441,STAR
4,1237655495977533571,198.80730,-1.448024,19.15370,19.34383,19.69506,19.96842,20.05856,-0.000006,-0.200451,-0.347111,-0.271725,-0.108460,STAR


In [12]:
comb.to_csv("sdss_dataset1.csv", index=False)

Let's also generate a shared test set:

In [13]:
test_set = pd.concat([stars[-1000:], galaxies[-1000:]])

In [14]:
test_set = test_set.sample(frac=1, replace=False).reset_index(drop=True)

In [15]:
test_set.head()

,objid,ra,dec,mag_u,mag_g,mag_r,mag_i,mag_z,spec_z,u_g_color,g_r_color,r_i_color,i_z_color,class
0,1237663783131218107,17.248432,-0.797119,21.42727,19.90164,19.43197,19.11375,18.58033,0.070120,1.750055,0.689732,0.371883,0.276276,GALAXY
1,1237678619567456496,4.893126,3.080620,22.26391,20.24837,18.86897,18.37562,17.88331,0.261890,1.931644,1.464487,0.524101,0.353077,GALAXY
2,1237656496174465529,329.603470,12.778041,20.29658,19.84159,18.97170,18.81909,18.79221,-0.001165,0.446840,0.868786,0.158445,0.017855,STAR
3,1237648702984290760,224.052880,-1.164559,20.30576,20.06680,20.15494,20.28310,20.48745,0.000329,0.298674,-0.114965,-0.123354,-0.245233,STAR
4,1237678619567456585,4.898836,3.127932,23.19452,21.11645,19.70312,19.14991,18.73902,0.266051,2.203650,1.469276,0.557816,0.320986,GALAXY


In [16]:
test_set.to_csv("sdss_testset1.csv")

## Neural Network Data

In [17]:
from astroML.datasets import fetch_sdss_galaxy_colors

In [25]:
NOBJECTS = 50000

In [31]:
query_text = ('\n'.join(("SELECT TOP %i" % NOBJECTS,
                         "  p.u, p.g, p.r, p.i, p.z, s.class, s.z AS redshift, s.zerr AS redshift_err",
                         "FROM PhotoObj AS p",
                         "  JOIN SpecObj AS s ON s.bestobjid = p.objid",
                         "WHERE ",
                         "  p.u BETWEEN 0 AND 19.6",
                         "  AND p.g BETWEEN 0 AND 20",
                         "  AND s.class <> 'UNKNOWN'",
                         "  AND s.class <> 'STAR'",
                         "  AND s.class <> 'SKY'",
                         "  AND s.class <> 'STAR_LATE'")))



In [32]:
data = SDSS.query_sql(query_text)

/astro/users/dhuppenk/.conda/envs/pytorch/lib/python3.8/site-packages/astroquery/sdss/core.py:862: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  arr = np.atleast_1d(np.genfromtxt(io.BytesIO(response.content),


In [34]:
data = data.to_pandas()
data["class"] = np.array(data["class"], dtype=str)




In [35]:
data.shape

(50000, 8)

In [30]:
data.head()

,u,g,r,i,z,class,z1,zerr
0,16.46322,15.49256,15.14610,15.00817,14.83415,GALAXY,0.012641,0.000006
1,17.23730,15.65943,14.76368,14.32308,13.97611,GALAXY,0.086139,0.000017
2,19.26072,17.97265,17.27236,16.88940,16.65260,GALAXY,0.103297,0.000021
3,18.87348,17.84478,17.35933,17.11581,16.95407,GALAXY,0.076046,0.000010
4,18.88721,17.34640,16.52588,16.08977,15.78733,GALAXY,0.075763,0.000009


In [36]:
training_data = data[:20000]

test_data = data[-4000:]

In [37]:
training_data.shape

(20000, 8)

In [38]:
test_data.shape

(4000, 8)

In [39]:
training_data.to_csv("sdss_dataset2.csv", index=False)

test_data.to_csv("sdss_testset2.csv", index=False)